# 7. LangSmith를 활용한 RAG 애플리케이션 평가



In [4]:
"""
환경 설정 및 API 키 구성
이 섹션에서는 LangSmith와 OpenAI API를 사용하기 위한 환경 변수를 설정합니다.
"""

# os 모듈: Python 표준 라이브러리로, 운영 체제와 상호작용하기 위한 인터페이스 제공
# 여기서는 환경 변수 설정에 사용됩니다
import os

# Google Colab 환경에서 사용자 데이터를 가져오기 위한 모듈 (현재는 주석 처리)
# from google.colab import userdata

# python-dotenv: .env 파일에서 환경 변수를 자동으로 로드하는 라이브러리
# 개발 환경에서 민감한 정보(API 키 등)를 안전하게 관리하기 위해 사용
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드
# .env 파일은 프로젝트 루트 디렉토리에 위치해야 하며, KEY=VALUE 형식으로 작성
load_dotenv()

# OpenAI API 키 설정
# OpenAI의 GPT 모델들을 사용하기 위한 인증 키
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")  # Colab 환경
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")  # 로컬 환경

# LangSmith 추적 설정
# LangSmith는 LangChain 애플리케이션의 디버깅, 테스팅, 모니터링을 위한 플랫폼
# LANGCHAIN_TRACING_V2: 추적 기능을 활성화 (true/false)
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# LangSmith API 엔드포인트 URL
# LangSmith 서비스에 접속하기 위한 API 서버 주소
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# LangSmith API 키 설정
# LangSmith 서비스 인증을 위한 API 키
# os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")  # Colab 환경
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")  # 로컬 환경

# LangSmith 프로젝트 이름
# 추적된 실행들을 구분하고 관리하기 위한 프로젝트 식별자
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

# Tavily API 키 설정
# Tavily는 웹 검색 API 서비스로, RAG 애플리케이션에서 실시간 정보 검색에 사용
# os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")  # Colab 환경
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")  # 로컬 환경

## 7.4. Ragas를 활용한 합성 테스트 데이터 생성


### 패키지 설치


In [5]:
!pip install langchain-core==0.2.30 langchain-openai==0.1.21 \
    langchain-community==0.2.12 GitPython==3.1.43 \
    langchain-chroma==0.1.2 chromadb==0.5.3 \
    ragas==0.1.14 nest-asyncio==1.6.0 pydantic==2.9.2 numpy==1.26.4

### 검색 대상 문서 로드


In [ ]:
"""
검색 대상 문서 로드
RAG 시스템의 지식 베이스로 사용할 문서들을 GitHub 저장소에서 로드합니다.
"""

# langchain_community: LangChain의 커뮤니티 통합 모듈
# document_loaders: 다양한 소스에서 문서를 로드하는 클래스들을 제공
# GitLoader: Git 저장소에서 파일을 로드하는 전문 로더
from langchain_community.document_loaders import GitLoader

# 파일 필터 함수: 로드할 파일을 선택하는 조건을 정의
# GitLoader는 이 함수를 사용해 특정 파일만 선택적으로 로드
def file_filter(file_path: str) -> bool:
    """
    로드할 파일을 필터링하는 함수
    
    Args:
        file_path (str): 확인할 파일의 경로
    
    Returns:
        bool: 파일을 로드할지 여부 (True면 로드, False면 스킵)
    """
    # Markdown 파일(.md)만 로드하도록 필터링
    # LangChain 문서는 주로 Markdown 형식으로 작성되어 있음
    return file_path.endswith(".md")


# GitLoader 인스턴스 생성 및 설정
loader = GitLoader(
    # GitHub 저장소의 클론 URL
    # LangChain 공식 저장소에서 문서를 가져옴
    clone_url="https://github.com/langchain-ai/langchain",
    
    # 로컬에 클론될 저장소 경로
    # 이미 존재하면 업데이트, 없으면 새로 클론
    repo_path="./langchain",
    
    # 특정 브랜치나 태그 지정
    # 특정 버전을 사용하여 일관성 있는 결과 보장
    branch="langchain==0.2.13",
    
    # 파일 필터 함수 지정
    # 위에서 정의한 file_filter 함수를 사용하여 .md 파일만 로드
    file_filter=file_filter,
)

# 문서 로드 실행
# load() 메서드는 지정된 조건에 맞는 모든 문서를 Document 객체 리스트로 반환
# 각 Document 객체는 page_content(텍스트 내용)와 metadata(파일 정보)를 포함
documents = loader.load()

# 로드된 문서 개수 출력
print(len(documents))

### Ragas를 활용한 합성 테스트 데이터 생성 구현


In [ ]:
"""
문서 메타데이터 처리
Ragas가 요구하는 형식에 맞게 메타데이터를 조정합니다.
"""

# 각 문서의 메타데이터를 수정
# Ragas는 'filename' 필드를 필요로 하므로 'source' 필드를 복사
for document in documents:
    # document.metadata는 딕셔너리로, 문서의 부가 정보를 저장
    # 'source': 원본 파일 경로 (GitLoader가 자동으로 설정)
    # 'filename': Ragas가 테스트 데이터 생성 시 참조하는 파일명 필드
    document.metadata["filename"] = document.metadata["source"]

#### 【주의】알려진 오류에 관해

아래 코드에서 gpt-4o를 사용할 경우 OpenAI API의 Usage tier에 따라 RateLimitError가 발생할 수 있다고 보고되었습니다.

OpenAI API의 Usage tier에 관해서는 공식 문서의 다음 페이지를 참조하세요.

https://platform.openai.com/docs/guides/rate-limits/usage-tiers

이 오류가 발생한 경우 다음 중 하나의 대응을 실시하세요.

1. 같은 Tier에서도 gpt-4o보다 레이트 리밋이 높은 gpt-4o-mini를 사용한다
   - 이 경우, 생성되는 합성 테스트 데이터의 품질이 낮아질 것으로 예상됩니다
2. 과금 등을 통해 Tier를 올린다
   - Tier 2에서는 RateLimitError가 발생하지 않는 것을 확인했습니다 (2024년 10월 31일 기준)

##### 2025/3/15 추가

LangChain 문서의 증가로 인해, gpt-4o-mini를 사용하더라도 Tier 1에서는 오류가 발생한다는 보고가 있습니다.

이 경우, GitHub에서 문서를 로드하는 부분에서 다음과 같이 작동이 확인된 버전인 `langchain==0.2.13`을 지정하도록 하세요.

```python
loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="langchain==0.2.13",
    file_filter=file_filter,
)
```


In [ ]:
# Ragas의 기능으로 합성 테스트 데이터 생성
import nest_asyncio
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

nest_asyncio.apply()

generator = TestsetGenerator.from_langchain(
    generator_llm=ChatOpenAI(model="gpt-4.1-mini"),
    critic_llm=ChatOpenAI(model="gpt-4.1-mini"),
    embeddings=OpenAIEmbeddings(),
)

testset = generator.generate_with_langchain_docs(
    documents,
    test_size=4, # 생성할 데이터 수를 4개로 설정
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

Generating: 100%|██████████| 4/4 [00:13<00:00,  3.37s/it]         


In [ ]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the Milvus vector database and how can...,[# langchain-milvus\n\nThis is a library integ...,The Milvus vector database is a vector databas...,simple,"[{'source': 'libs/partners/milvus/README.md', ...",True
1,What are the key features of Airtable that dif...,[# Airtable\n\n>[Airtable](https://en.wikipedi...,Airtable is a spreadsheet-database hybrid that...,simple,[{'source': 'docs/docs/integrations/providers/...,True
2,How to start & access vertexai-chuck-norris lo...,[\n# vertexai-chuck-norris\n\nThis template ma...,To start the vertexai-chuck-norris local serve...,reasoning,[{'source': 'templates/vertexai-chuck-norris/R...,True
3,How does Milvus hybrid search in langchain-mil...,[# langchain-milvus\n\nThis is a library integ...,The context does not provide information on ho...,multi_context,"[{'source': 'libs/partners/milvus/README.md', ...",True


### LangSmith의 Dataset 생성


In [ ]:
from langsmith import Client

dataset_name = "agent-book"

client = Client()

if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(dataset_name=dataset_name)

### 합성 테스트 데이터 저장


In [ ]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
    inputs.append(
        {
            "question": testset_record.question,
        }
    )
    outputs.append(
        {
            "contexts": testset_record.contexts,
            "ground_truth": testset_record.ground_truth,
        }
    )
    metadatas.append(
        {
            "source": testset_record.metadata[0]["source"],
            "evolution_type": testset_record.evolution_type,
        }
    )

In [ ]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    metadata=metadatas,
    dataset_id=dataset.id,
)

## 7.5. LangSmith와 Ragas를 활용한 오프라인 평가 구현


### 커스텀 Evaluator 구현


In [ ]:
from typing import Any

from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel
from langsmith.schemas import Example, Run
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM


class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric

        # LLM과 Embeddings을 Metric에 설정
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]

        # Ragas의 평가 메트릭의 score 메서드로 점수 산출
        score = self.metric.score(
            {
                "question": example.inputs["question"],
                "answer": run.outputs["answer"],
                "contexts": context_strs,
                "ground_truth": example.outputs["ground_truth"],
            },
        )
        return {"key": self.metric.name, "score": score}

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import answer_relevancy, context_precision

metrics = [context_precision, answer_relevancy]

llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

evaluators = [
    RagasMetricEvaluator(metric, llm, embeddings).evaluate
    for metric in metrics
]

### 추론 함수 구현


In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 고려해 질문에 답하세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

NameError: name 'db' is not defined

In [ ]:
def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    question = inputs["question"]
    output = chain.invoke(question)
    return {
        "contexts": output["context"],
        "answer": output["answer"],
    }

### 오프라인 평가 구현·실행


In [ ]:
from langsmith.evaluation import evaluate

evaluate(
    predict,
    data="agent-book",
    evaluators=evaluators,
)

View the evaluation results for experiment: 'enchanted-case-10' at:
https://smith.langchain.com/o/8e65c94f-633f-413a-a8ec-eb44d58ed5fc/datasets/225de56f-7e22-406f-9a9d-5ed3f028b809/compare?selectedSessions=71209035-0e1e-498e-a2b6-1eef77d05235




0it [00:00, ?it/s]Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
4it [00:33,  8.48s/it]


,inputs.question,outputs.contexts,outputs.answer,error,reference.contexts,reference.ground_truth,feedback.context_precision,feedback.answer_relevancy,execution_time,example_id,id
0,How does Milvus hybrid search in langchain-mil...,[page_content='# langchain-milvus\n\nThis is a...,The context provided does not contain specific...,None,[# langchain-milvus\n\nThis is a library integ...,The context does not provide information on ho...,0.000000,0.000000,3.617417,0a4c6d24-ca36-445f-904f-929e8ad64423,60668606-de61-4fb1-8fba-e4f19b0aa97d
1,What is the Milvus vector database and how can...,[page_content='# langchain-milvus\n\nThis is a...,The Milvus vector database is a high-performan...,None,[# langchain-milvus\n\nThis is a library integ...,The Milvus vector database is a vector databas...,1.000000,0.965116,6.586956,91bc5f59-fcef-4c8a-af17-0cc36b8e4a1f,4044b452-59e6-47e1-827e-d49c45d82887
2,How to start & access vertexai-chuck-norris lo...,[page_content='\n# vertexai-chuck-norris\n\nTh...,To start and access the `vertexai-chuck-norris...,None,[\n# vertexai-chuck-norris\n\nThis template ma...,To start the vertexai-chuck-norris local serve...,0.805556,0.965608,11.147836,b96c61f0-12b0-4bb5-a68b-723e9d521e01,b3c14c28-a8e6-4c87-8bd8-70d0a7858f29
3,What are the key features of Airtable that dif...,[page_content='# Airtable\n\n>[Airtable](https...,Airtable differentiates itself from a traditio...,None,[# Airtable\n\n>[Airtable](https://en.wikipedi...,Airtable is a spreadsheet-database hybrid that...,1.000000,0.939072,8.094627,361d850c-c54b-4c10-b8ba-25ba08d417c7,84e4eab9-e9d9-4521-9078-6f01a9805f96


## LangSmith를 활용한 온라인 평가 구현


### 피드백 버튼을 표시하는 함수 구현


In [ ]:
!pip install uuid ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]3 [ipywidgets]


In [ ]:
from uuid import UUID

import ipywidgets as widgets
from IPython.display import display
from langsmith import Client


def display_feedback_buttons(run_id: UUID) -> None:
    # Good 버튼과 Bad 버튼 준비
    good_button = widgets.Button(
        description="Good",
        button_style="success",
        icon="thumbs-up",
    )
    bad_button = widgets.Button(
        description="Bad",
        button_style="danger",
        icon="thumbs-down",
    )

    # 클릭 시 실행될 함수 정의
    def on_button_clicked(button: widgets.Button) -> None:
        if button == good_button:
            score = 1
        elif button == bad_button:
            score = 0
        else:
            raise ValueError(f"Unknown button: {button}")

        client = Client()
        client.create_feedback(run_id=run_id, key="thumbs", score=score)
        print("피드백을 전송했습니다")

    # 버튼 클릭 시 on_button_clicked 함수 실행
    good_button.on_click(on_button_clicked)
    bad_button.on_click(on_button_clicked)

    # 버튼 표시
    display(good_button, bad_button)

### 피드백 버튼 표시


In [ ]:
from langchain_core.tracers.context import collect_runs

# LangSmith의 트레이스 ID(Run ID)를 가져오기 위해 collect_runs 함수 사용
with collect_runs() as runs_cb:
    output = chain.invoke("LangChain의 개요를 알려줘")
    print(output["answer"])
    run_id = runs_cb.traced_runs[0].id

display_feedback_buttons(run_id)

NameError: name 'chain' is not defined